<h1 style = "font-size:3rem;color:darkcyan"> Implementing autoencoders</h1>


Implementing an deep convoluational autoencoder class with mirrored encoder and decoder components. 
<i>Based on the work of Valerio Velardo.</i>

In [49]:
# importing libraries
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, ReLU, \
BatchNormalization, Flatten, Dense, Reshape, Conv2DTranspose, \
Activation

In [60]:
class Autoencoder:
    
    def __init__(self, 
                input_shape,
                conv_filters,
                conv_kernels,
                conv_strides,
                latent_space_dim):
        
        self.input_shape = input_shape 
        self.conv_filters = conv_filters
        self.conv_kernels = conv_kernels
        self.conv_strides = conv_strides
        self.latent_space_dim = latent_space_dim
        
        self.encoder = None
        self.decoder = None
        self.model = None
        
        self._num_conv_layers = len(conv_filters)
        self._shape_before_bottleneck = None
        
        self._build()
        
    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        
    def _build(self):
        self._build_encoder()
        self._build_decoder()
        #self._build_autoencoder() 
        
    def _build_encoder(self):
        encoder_input = self._add_encoder_input()
        conv_layers = self._add_conv_layers(encoder_input)
        bottleneck = self._add_bottleneck(conv_layers)
        self.encoder = Model(encoder_input, bottleneck, name = 'encoder')
    
    def _add_encoder_input(self):
        return Input(shape = self.input_shape, name = 'encoder_input')
    
    def _add_conv_layers(self, encoder_input):
        layer_graph = encoder_input
        for i in range(self._num_conv_layers):
            layer_graph = self._add_conv_layer(i, layer_graph)
        return layer_graph
    
    def _add_conv_layer(self, layer_index, layer_graph):
        # conv2D + ReLu + batch normalization
        
        current_layer = layer_index + 1
        conv_layer = Conv2D(
            filters = self.conv_filters[layer_index],
            kernel_size = self.conv_kernels[layer_index],
            strides = self.conv_strides[layer_index],
            padding = 'same',
            name = f'encoder_conv_layer_{current_layer}'
        )
        
        layer_graph = conv_layer(layer_graph)
        layer_graph = ReLU(name=f'encoder_relu_{current_layer}')(layer_graph)
        layer_graph = BatchNormalization(name=f'encoder_bn_{current_layer}')(layer_graph)
        
        return layer_graph
    
    def _add_bottleneck(self, layer_graph): 
        # save shape for decoding
        self._shape_before_bottleneck = K.int_shape(layer_graph)[1:]
        # flatten data and add Dense layer (bottleneck)
        layer_graph = Flatten()(layer_graph)
        layer_graph = Dense(self.latent_space_dim, name = 'encoder_output')(layer_graph)
        return layer_graph
    
    def _build_decoder(self):
        decoder_input = self._add_decoder_input()
        dense_layer = self._add_dense_layer(decoder_input)
        reshape_layer = self._add_reshape_layer(dense_layer)
        conv_transpose_layers = self._add_conv_transpose_layers(reshape_layer)
        decoder_output = self._add_decoder_output(conv_transpose_layers)
        self.decoder = Model(decoder_input, decoder_output, name = 'decoder')

    def _add_decoder_input(self):
        return Input(shape = self.latent_space_dim, name = 'decoder_input')
    
    def _add_dense_layer(self, decoder_input):
        num_neurons = np.prod(self._shape_before_bottleneck)
        return Dense(num_neurons, name = 'decoder_dense')(decoder_input)
       
    def _add_reshape_layer(self, dense_layer):
        return Reshape(self._shape_before_bottleneck)(dense_layer)
    
    def _add_conv_transpose_layers(self, layer_graph):
        for i in reversed(range(1, self._num_conv_layers)): # ignore input layer
            layer_graph = self._add_conv_transpose_layer(i, layer_graph)
        return layer_graph
    
    def _add_conv_transpose_layer(self, layer_index, layer_graph):
        layer_num = self._num_conv_layers - layer_index
        conv_transpose_layer = Conv2DTranspose(
            filters = self.conv_filters[layer_index],
            kernel_size = self.conv_kernels[layer_index],
            strides = self.conv_strides[layer_index],
            padding = 'same',
            name = f'decoder_conv_transpose_layer_{layer_num}'
        )
        
        layer_graph = conv_transpose_layer(layer_graph)
        layer_graph = ReLU(name = f'decoder_ReLU_{layer_num}')(layer_graph)
        layer_graph = BatchNormalization(name = f'decoder_bn_{layer_num}')(layer_graph)
        
        return layer_graph
    
    def _add_decoder_output(self, layer_graph):
        conv_transpose_layer = Conv2DTranspose(
            filters = 1,
            kernel_size = self.conv_kernels[0],
            strides = self.conv_strides[0],
            padding = 'same',
            name = f'decoder_conv_transpose_layer_{self._num_conv_layers}'
        ) 
        
        layer_graph = conv_transpose_layer(layer_graph)
        output_layer = Activation('sigmoid', name = 'output_sigmoid_layer')(layer_graph)
        return output_layer
        
        
            
            

In [61]:
autoencoder = Autoencoder(
    input_shape = (28,28,1),
    conv_filters = (32,64,64,64),
    conv_kernels = (3,3,3,3),
    conv_strides = (1,2,2,1),
    latent_space_dim = 2
)

In [62]:
autoencoder.summary()


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_layer_1 (Conv2  (None, 28, 28, 32)       320       
 D)                                                              
                                                                 
 encoder_relu_1 (ReLU)       (None, 28, 28, 32)        0         
                                                                 
 encoder_bn_1 (BatchNormaliz  (None, 28, 28, 32)       128       
 ation)                                                          
                                                                 
 encoder_conv_layer_2 (Conv2  (None, 14, 14, 64)       18496     
 D)                                                              
                                                           